# Data prep

In [1]:
import numpy as np

with open('university_large.ass', 'r') as file:
    lines = [line.rstrip() for line in file.readlines()]

In [2]:
parameters = dict()
matrix = list()

for line in lines:
    if line.startswith('@'):
        if '{' in line:
            parameter_name = line.split('{')[0].strip('{}@ ')
            parameter_value = line.split('{')[1].strip('{}@ ').split(',')
            
            parameters[parameter_name] = parameter_value
        
    elif ('0' in line or '1' in line) and ',' in line:
        matrix.append([int(x) for x in line.split(',')])
        
matrix = np.array(matrix)

assert len(parameters['dimension user']) == matrix.shape[0]
assert len(parameters['dimension permission']) == matrix.shape[1]

print(matrix.shape)

(493, 56)


# Greedy algorithm

Abstract:

This algorithm starts with least permissions user and calculates his role. Then other users with same permissions are given this role. 
Its greed allows to effectively minimize number of roles

In [3]:
def calculate_coverage(permissions_vector, cover_vector):
    num_uncovered_permissions = 0
    
    for i in range(len(permissions_vector)):
        if permissions_vector[i] == 1 and cover_vector[i] == 0:
            num_uncovered_permissions += 1
            
    return num_uncovered_permissions

def add_permissions(cover_vector, permissions_vector):
    for i in range(len(permissions_vector)):
        if permissions_vector[i] == 1:
            cover_vector[i] = 1
            
    return cover_vector

def has_permissions_subset(user_permissions, permissions_subset):
    for i in range(len(permissions_subset)):
        if permissions_subset[i] == 1 and user_permissions[i] != 1:
            return False
    return True

In [11]:
covered_permissions = [0]*matrix.shape[1]
user_groups = list()
user_groups_permissions = list()

iterations = 0

while 0 in covered_permissions:
    iterations += 1
    
    min_uncovered_permissions = matrix.shape[1]
    min_uncovered_permissions_user_index = 0

    for i in range(matrix.shape[0]):
        num_uncovered_permissions = calculate_coverage(matrix[i], covered_permissions)
        if num_uncovered_permissions < min_uncovered_permissions and num_uncovered_permissions != 0:
            min_uncovered_permissions = num_uncovered_permissions
            min_uncovered_permissions_user_index = i
        
    permissions_subset = matrix[min_uncovered_permissions_user_index]
    user_group = [min_uncovered_permissions_user_index]
    
    for i in range(matrix.shape[0]):
        if i == min_uncovered_permissions_user_index:
            continue
        
        if has_permissions_subset(matrix[i], permissions_subset):
            user_group.append(i)
            
    covered_permissions = add_permissions(covered_permissions, permissions_subset)
    user_groups.append(user_group)
    user_groups_permissions.append(permissions_subset)
    
    print(f'Iter: #{iterations}, users in group: {len(user_group)}, user group permissions: {sum(permissions_subset)}')

Iter: #1, users in group: 5, user group permissions: 2
Iter: #2, users in group: 5, user group permissions: 2
Iter: #3, users in group: 9, user group permissions: 3
Iter: #4, users in group: 5, user group permissions: 3
Iter: #5, users in group: 300, user group permissions: 7
Iter: #6, users in group: 30, user group permissions: 8
Iter: #7, users in group: 20, user group permissions: 9
Iter: #8, users in group: 9, user group permissions: 11
Iter: #9, users in group: 40, user group permissions: 9
Iter: #10, users in group: 5, user group permissions: 9
Iter: #11, users in group: 73, user group permissions: 9
Iter: #12, users in group: 1, user group permissions: 10
Iter: #13, users in group: 4, user group permissions: 11
Iter: #14, users in group: 10, user group permissions: 21
Iter: #15, users in group: 6, user group permissions: 26
Iter: #16, users in group: 3, user group permissions: 30
Iter: #17, users in group: 1, user group permissions: 40


In [12]:
user_groups_permissions

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1, 1, 1